# Thesis

## Install Library

In [ ]:
!pip install roboflow
!pip install ultralytics

## Import Library

In [ ]:
import time
import os
import csv
import shutil
import pandas as pd
import matplotlib.pyplot as plt
from roboflow import Roboflow
from ultralytics import YOLO
from ultralytics.data.annotator import auto_annotate

## Download Dataset

In [ ]:
# Dataset Object Detection
rf = Roboflow(api_key="m1DZYxXInBZk9zMy3jDT")
project = rf.workspace("abiya").project("plant-pathology-anotation")
version = project.version(7)
dataset = version.download("yolov11", location="dataset/objectDetection/")

In [ ]:
# Dataset Mask Lesi
rf = Roboflow(api_key="m1DZYxXInBZk9zMy3jDT")
project = rf.workspace("abiya").project("plant-pathology-anotation")
version = project.version(10)
dataset = version.download("yolov11", location="dataset/maskLesi/")

## Variabel Global

In [ ]:
# Variabel Global
epochs = 100
imgsz = 640
model_variants_objectDetection = ['n', 's', 'm']
model_variants_maskDaun = ['n', 's', 'm']
model_variants_maskLesi = ['n', 's', 'm']
# ['n', 's', 'm', 'l']

## Training Object Detection

In [ ]:
# Global Variables
dataset_objectDetection = "dataset/objectDetection/data.yaml"
output_csv_objectDetection = "results/objectDetection/csv/"
output_runs_objectDetection = "results/objectDetection/runs/"
output_plot_objectDetection = "results/objectDetection/plot/"

# Memastikan folder output ada
os.makedirs(output_csv_objectDetection, exist_ok=True)
os.makedirs(output_runs_objectDetection, exist_ok=True)
os.makedirs(output_plot_objectDetection, exist_ok=True)

In [ ]:
# Inisialisasi file ringkasan
summary_train_file = os.path.join(output_csv_objectDetection, "final_train_summary.csv")
summary_val_file = os.path.join(output_csv_objectDetection, "final_val_summary.csv")

# Header ringkasan
train_header = [
    "variant", "train_time_h", "train_time_m", "train_time_s"
]
val_header = [
    "variant", "precision", "recall", "mAP50", "mAP50-95",
]

# Inisialisasi file ringkasan
with open(summary_train_file, mode="w", newline="") as f_train, \
     open(summary_val_file, mode="w", newline="") as f_val:
    csv.writer(f_train).writerow(train_header)
    csv.writer(f_val).writerow(val_header)

In [ ]:
# Loop tiap varian model
for variant in model_variants_objectDetection:
    model_name = f"yolo11{variant}.pt"
    train_name = f"train_{variant}"
    project_path = f"{output_runs_objectDetection}detect/{train_name}"
    model = YOLO(model_name)

    # Train
    start_time = time.time()
    results = model.train(
        data=dataset_objectDetection,
        epochs=epochs,
        imgsz=imgsz,
        project=f"{output_runs_objectDetection}/detect",
        name=train_name,
        verbose=False
    )
    end_time = time.time()

    # Simpan waktu training
    elapsed = end_time - start_time
    hours = int(elapsed // 3600)
    minutes = int((elapsed % 3600) // 60)
    seconds = int(elapsed % 60)

    # Simpan ke summary_train_file
    with open(summary_train_file, mode="a", newline="") as f_train:
        csv.writer(f_train).writerow([variant, hours, minutes, seconds])

    # === Simpan log setiap epoch ===
    results_csv_path = os.path.join(project_path, "results.csv")
    if os.path.exists(results_csv_path):
        df = pd.read_csv(results_csv_path)
        df.to_csv(os.path.join(output_csv_objectDetection, f"train_logs_{variant}.csv"), index=False)

    # === Evaluasi / Validasi ===
    best_model_path = os.path.join(project_path, "weights", "best.pt")
    model = YOLO(best_model_path)
    val_name = f"val_{variant}"
    metrics = model.val(
        data=dataset_objectDetection,
        imgsz=imgsz,
        split='test',
        project=f"{output_runs_objectDetection}/detect",
        name=val_name,
        verbose=False
    )

    # Simpan ke summary_val_file
    with open(summary_val_file, mode="a", newline="") as f_val:
        writer = csv.writer(f_val)
        results_dict = metrics.results_dict
        writer.writerow([
            variant,
            round(results_dict['metrics/precision(B)'], 4),
            round(results_dict['metrics/recall(B)'], 4),
            round(results_dict['metrics/mAP50(B)'], 4),
            round(results_dict['metrics/mAP50-95(B)'], 4),
        ])


In [ ]:
# Perbandingan Metrik antar Varian
metrics = {
    "metrics/mAP50(B)": "mAP@0.5",
    "metrics/mAP50-95(B)": "mAP@0.5-0.95",
    "time": "time",
}

# Buat dictionary file
files = {v: os.path.join(output_csv_objectDetection, f"train_logs_{v}.csv") for v in model_variants_objectDetection}

# Loop setiap metrik dan buat plot perbandingan
for metric_col, metric_label in metrics.items():
    plt.figure()
    for variant, file_path in files.items():
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            if metric_col in df.columns:
                plt.plot(df["epoch"], df[metric_col], label=f"YOLOv11-{variant}")

    plt.xlabel("Epoch")
    plt.ylabel(metric_label)
    plt.title(f"Perbandingan {metric_label} antar Varian")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    safe_name = metric_col.replace("/", "_")
    plt.savefig(os.path.join(output_plot_objectDetection, f"compare_{safe_name}.png"))
    plt.close()

## Training Mask Daun

In [ ]:
# Variabel Global
output_runs_maskDaun = "results/maskDaun/runs/"
output_csv_maskDaun = "results/maskDaun/csv/"
output_plot_maskDaun = "results/maskDaun/plot/"
original_dataset = "dataset/objectDetection"
maskDaun_dataset = "dataset/maskDaun"
sam2_model = "sam2_b.pt"
best_model_path = "results/objectDetection/runs/detect/train_m/weights/best.pt"

# Memastikan folder output ada
os.makedirs(output_runs_maskDaun, exist_ok=True)
os.makedirs(output_csv_maskDaun, exist_ok=True)
os.makedirs(output_plot_maskDaun, exist_ok=True)

In [ ]:
# Copy Folder Dataset
shutil.copytree(src=original_dataset, dst=maskDaun_dataset, dirs_exist_ok=True)

# Hapus folder labels pada train, valid, test
splits = ["train", "valid", "test"]
for split in splits:
    label_path = os.path.join(maskDaun_dataset, split, "labels")
    if os.path.exists(label_path):
        shutil.rmtree(label_path)

In [ ]:
# Auto Annotate

# train
auto_annotate(data=f"{maskDaun_dataset}/train/images/", det_model=best_model_path, sam_model=sam2_model, output_dir=f"{maskDaun_dataset}/train/labels/")

# valid
auto_annotate(data=f"{maskDaun_dataset}/valid/images/", det_model=best_model_path, sam_model=sam2_model, output_dir=f"{maskDaun_dataset}/valid/labels/")

# test
auto_annotate(data=f"{maskDaun_dataset}/test/images/", det_model=best_model_path, sam_model=sam2_model, output_dir=f"{maskDaun_dataset}/test/labels/")

In [ ]:
# ===== File CSV Ringkasan Evaluasi =====
summary_csv = os.path.join(output_csv_maskDaun, "summary_segmentation_metrics.csv")
with open(summary_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["variant", "mAP_box", "mAP_mask"])

# ===== Loop Training dan Validasi =====
for variant in model_variants_maskDaun:
    model_name = f"yolo11{variant}-seg.pt"
    run_name = f"train_maskDaun_{variant}"
    project_dir = os.path.join(output_runs_maskDaun, run_name)

    # Load dan latih model
    model = YOLO(model_name)
    results = model.train(
        data=f"{maskDaun_dataset}/data.yaml",
        epochs=epochs,
        imgsz=imgsz,
        project=output_runs_maskDaun,
        name=run_name,
        verbose=False
    )

    # Simpan CSV log training per epoch
    results_csv = os.path.join(project_dir, "results.csv")
    if os.path.exists(results_csv):
        df = pd.read_csv(results_csv)
        df.to_csv(os.path.join(output_csv_maskDaun, f"train_logs_seg_{variant}.csv"), index=False)

    # Validasi terhadap split test
    best_model_path = os.path.join(project_dir, "weights", "best.pt")
    model = YOLO(best_model_path)
    val_name = f"val_maskDaun_{variant}"
    metrics = model.val(
        data=f"{maskDaun_dataset}/data.yaml",
        imgsz=imgsz,
        split='test',
        project=output_runs_maskDaun,
        name=val_name,
        verbose=False
    )

    # Simpan mAP ke summary CSV
    results_dict = metrics.results_dict
    with open(summary_csv, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            variant,
            round(results_dict['metrics/mAP50(B)'], 4),
            round(results_dict['metrics/mAP50(M)'], 4)
        ])

In [ ]:
metric_labels = {
    "metrics/mAP50(B)": "mAP@0.5",
    "metrics/mAP50-95(B)": "mAP@0.5-0.95",
    "metrics/mAP50(M)": "mAP@0.5-0.95",
    "metrics/mAP50-95(M)": "mAP@0.5-0.95",
    "time": "time",
}

# === Plot log per epoch ===
for metric_col, label in metric_labels.items():
    plt.figure()
    for v in model_variants_maskDaun:
        file_path = os.path.join(output_csv_maskDaun, f"train_logs_seg_{v}.csv")
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            if metric_col in df.columns:
                plt.plot(df["epoch"], df[metric_col], label=f"YOLOv11-{v}")
    plt.xlabel("Epoch")
    plt.ylabel(label)
    plt.title(f"Perbandingan {label} antar Varian (Segmentation)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    safe_name = metric_col.replace("/", "_")
    plt.savefig(os.path.join(output_plot_maskDaun, f"compare_seg_{safe_name}.png"))
    plt.close()

# === Plot summary mAP Box dan mAP Mask ===
summary_file = os.path.join(output_csv_maskDaun, "summary_segmentation_metrics.csv")
if os.path.exists(summary_file):
    df = pd.read_csv(summary_file)
    for col, label in zip(["mAP_box", "mAP_mask"], ["mAP Box", "mAP Mask"]):
        plt.figure()
        plt.bar(df["variant"], df[col])
        plt.title(f"{label} per Varian (Segmentasi)")
        plt.ylabel(label)
        plt.xlabel("YOLOv8 Variant")
        plt.tight_layout()
        plt.savefig(os.path.join(output_plot_maskDaun, f"summary_{col}.png"))
        plt.close()

## Training Mask Lesi

In [ ]:
# Variabel Global
dataset_maskLesi = "dataset/maskLesi/data.yaml"
output_csv_maskLesi = "results/maskLesi/csv/"
output_runs_maskLesi = "results/maskLesi/runs/"
output_plot_maskLesi = "results/maskLesi/plot/"

# Buat folder jika belum ada
os.makedirs(output_csv_maskLesi, exist_ok=True)
os.makedirs(output_runs_maskLesi, exist_ok=True)
os.makedirs(output_plot_maskLesi, exist_ok=True)

In [ ]:
# === Inisialisasi Summary CSV ===
summary_csv = os.path.join(output_csv_maskLesi, "summary_segmentation_metrics.csv")
with open(summary_csv, "w", newline="") as f:
    csv.writer(f).writerow(["variant", "mAP_box", "mAP_mask"])

# === Training dan Validasi Loop ===
for variant in model_variants_maskLesi:
    model_path = f"yolo11{variant}-seg.pt"
    run_name = f"train_maskLesi_{variant}"
    run_path = os.path.join(output_runs_maskLesi, run_name)

    # Load dan latih model
    model = YOLO(model_path)
    results = model.train(
        data=dataset_maskLesi,
        epochs=epochs,
        imgsz=imgsz,
        project=output_runs_maskLesi,
        name=run_name,
        verbose=False
    )

    # Simpan log training per epoch
    results_csv = os.path.join(run_path, "results.csv")
    if os.path.exists(results_csv):
        df = pd.read_csv(results_csv)
        df.to_csv(os.path.join(output_csv_maskLesi, f"train_logs_seg_{variant}.csv"), index=False)

    # Validasi dan ambil metrik
    best_model_path = os.path.join(run_path, "weights", "best.pt")
    model = YOLO(best_model_path)
    metrics = model.val(
        data=dataset_maskLesi, 
        imgsz=imgsz, 
        project=output_runs_maskLesi,
        name=f"val_maskLesi_{variant}",
        split='test', 
        verbose=False)
    results_dict = metrics.results_dict

    # Simpan ringkasan mAP
    with open(summary_csv, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            variant,
            round(results_dict['metrics/mAP50(B)'], 4),
            round(results_dict['metrics/mAP50(M)'], 4)
        ])

In [ ]:
# === Visualisasi Otomatis ===
metric_labels = {
    "metrics/mAP50(B)": "mAP@0.5",
    "metrics/mAP50-95(B)": "mAP@0.5-0.95",
    "metrics/mAP50(M)": "mAP@0.5",
    "metrics/mAP50-95(M)": "mAP@0.5-0.95",
    "time": "time",
}

# Plot log per epoch
for metric_col, label in metric_labels.items():
    plt.figure()
    for variant in model_variants_maskLesi:
        log_path = os.path.join(output_csv_maskLesi, f"train_logs_seg_{variant}.csv")
        if os.path.exists(log_path):
            df = pd.read_csv(log_path)
            if metric_col in df.columns:
                plt.plot(df["epoch"], df[metric_col], label=f"YOLOv11-{variant}")
    plt.xlabel("Epoch")
    plt.ylabel(label)
    plt.title(f"{label} per Epoch (Mask Lesi)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    safe_name = metric_col.replace("/", "_")
    plt.savefig(os.path.join(output_plot_maskLesi, f"compare_{safe_name}.png"))
    plt.close()

# Plot summary mAP
summary_df = pd.read_csv(summary_csv)
for col, label in zip(["mAP_box", "mAP_mask"], ["mAP Box", "mAP Mask"]):
    plt.figure()
    plt.bar(summary_df["variant"], summary_df[col])
    plt.title(f"{label} per Varian (Mask Lesi)")
    plt.ylabel(label)
    plt.xlabel("YOLOv8 Variant")
    plt.tight_layout()
    plt.savefig(os.path.join(output_plot_maskLesi, f"summary_{col}.png"))
    plt.close()